#### Importing python libraries

In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
import warnings
warnings.filterwarnings("ignore")

import sys
#import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py as h5
import sklearn
from sklearn.multioutput import MultiOutputClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, precision_recall_curve, accuracy_score, confusion_matrix  
from sklearn.metrics import average_precision_score
import pickle
import keras
from keras.models import load_model

import util

print("Python version:\n{}\n".format(sys.version))
print("matplotlib version: {}".format(matplotlib.__version__))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("sklearn version: {}".format(sklearn.__version__))
print("keras version: {}".format(keras.__version__))


Using TensorFlow backend.


Python version:
3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]

matplotlib version: 3.1.3
pandas version: 0.24.2
numpy version: 1.18.3
sklearn version: 0.22.1
keras version: 2.2.4


## Loading data: Held out test set

In [2]:
path = '/path/to/data/'

data_test = np.load(path + 'test.npz', allow_pickle=True)
test_x = data_test['signal']
test_qa = data_test['qa_label']
test_r = data_test['rhythm']
test_p = pd.DataFrame(data_test['parameters'])
print(test_x.shape)
print(test_qa.shape)
print(test_r.shape)
print(test_p.shape)
test_p.rename(index=str, columns={0:'timestamp', 
                                  1:'stream', 
                                  2:'ID'}, inplace=True)

(17617, 800, 1)
(17617, 3)
(17617, 2)
(17617, 3)


## VGG model : single-task


In [3]:
path = '/path/to/model/'
vgg = load_model(path + 'VGG_singletask.h5')


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


#### VGG Single-task 


In [4]:
# weighted macro-average across all indivduals

test_metrics = util.collecting_individual_metrics_singletask(vgg, test_x, test_p, test_r, out_message=False)
test_metrics = pd.DataFrame.from_dict(test_metrics).T.rename(columns={0:'TPR', 1:'TNR', 2:'FPR', 3:'FNR', 4:"total_samples"})

for m in ['TPR', 'TNR', 'FPR', 'FNR']:
    metric_wmu = np.average(test_metrics[m][~test_metrics[m].isna()], weights=test_metrics['total_samples'][~test_metrics[m].isna()])
    print('%s: %0.2f' % (m, metric_wmu))
    
# PPV, NPV and F1

episode_m = util.episode_metrics_singletask(vgg, test_x, test_p, test_r, out_message=False)
episode_metrics = pd.DataFrame(episode_m).T
episode_metrics.rename(columns={0:'TPR', 1:'TNR', 2:'PPV', 3:'NPV', 4:"FPR", 5:'FNR', 6:'F1', 7:'total_samples'}, inplace=True)
for m in ['PPV', 'NPV', 'F1']:
    print('%s: %0.2f' % (m, episode_metrics[m]))


TPR: 0.92
TNR: 0.71
FPR: 0.29
FNR: 0.08
PPV: 0.50
NPV: 0.96
F1: 0.64


## DeepBeat Single task (no pretrain CDAE + AF task)

#### Loading non pretrained DeepBeat single task model

In [5]:
path = '/path/to/model/'
model_reint = load_model(path + 'deepbeat_singletask_nopretrain.h5')


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


#### DeepBeat Single task

In [6]:
# weighted macro-average across all indivduals

test_metrics = util.collecting_individual_metrics_singletask(model_reint, test_x, test_p, test_r, out_message=False)
test_metrics = pd.DataFrame.from_dict(test_metrics).T.rename(columns={0:'TPR', 1:'TNR', 2:'FPR', 3:'FNR', 4:"total_samples"})


for m in ['TPR', 'TNR', 'FPR', 'FNR']:
    metric_wmu = np.average(test_metrics[m][~test_metrics[m].isna()], weights=test_metrics['total_samples'][~test_metrics[m].isna()])
    print('%s: %0.2f' % (m, metric_wmu))
    
# PPV, NPV and F1

episode_m = util.episode_metrics_singletask(model_reint, test_x, test_p, test_r, out_message=False)
episode_metrics = pd.DataFrame(episode_m).T
episode_metrics.rename(columns={0:'TPR', 1:'TNR', 2:'PPV', 3:'NPV', 4:"FPR", 5:'FNR', 6:'F1', 7:'total_samples'}, inplace=True)
for m in ['PPV', 'NPV', 'F1']:
    print('%s: %0.2f' % (m, episode_metrics[m]))


TPR: 0.49
TNR: 0.90
FPR: 0.10
FNR: 0.51
PPV: 0.60
NPV: 0.85
F1: 0.54


## DeepBeat Single task (pretrain with CDAE + AF)

#### Loading pretrained DeepBeat single task model

In [7]:
path_to_model ='/path/to/model/'
model_name = 'deepbeat_singletask_pretrained.h5'
deepbeat_st = load_model(path_to_model + model_name) 

#### DeepBeat Single task

In [8]:
# weighted macro-average across all indivduals

test_metrics = util.collecting_individual_metrics_singletask(deepbeat_st, test_x, test_p, test_r, out_message=False)
test_metrics = pd.DataFrame.from_dict(test_metrics).T.rename(columns={0:'TPR', 1:'TNR', 2:'FPR', 3:'FNR', 4:"total_samples"})


for m in ['TPR', 'TNR', 'FPR', 'FNR']:
    metric_wmu = np.average(test_metrics[m][~test_metrics[m].isna()], weights=test_metrics['total_samples'][~test_metrics[m].isna()])
    print('%s: %0.2f' % (m, metric_wmu))
    
# PPV, NPV and F1

episode_m = util.episode_metrics_singletask(deepbeat_st, test_x, test_p, test_r, out_message=False)
episode_metrics = pd.DataFrame(episode_m).T
episode_metrics.rename(columns={0:'TPR', 1:'TNR', 2:'PPV', 3:'NPV', 4:"FPR", 5:'FNR', 6:'F1', 7:'total_samples'}, inplace=True)
for m in ['PPV', 'NPV', 'F1']:
    print('%s: %0.2f' % (m, episode_metrics[m]))

TPR: 0.52
TNR: 0.88
FPR: 0.12
FNR: 0.48
PPV: 0.59
NPV: 0.85
F1: 0.56


## DeepBeat Multi-task (no pretrain CDAE + AF + Excellent QA)

#### Loading non pretrained DeepBeat model

In [9]:
path_to_model ='/path/to/model/'
model_reint = load_model(path_to_model + 'deepbeat_no_pretraining.h5')

#### DeepBeat Multi-task (no pretrain CDAE + AF + Excellent QA) 

In [10]:
## QA results

predictions_qa, predictions_r = model_reint.predict(test_x)
predictions_QA = np.argmax(predictions_qa, axis=1)

#print(classification_report(np.argmax(test_qa, axis=1), predictions_QA))


excellent_qa_indx = np.where(predictions_QA==2)[0]
x_test_excellent = test_x[excellent_qa_indx,:]
p_test_excellent = test_p.iloc[excellent_qa_indx,:]
rhythm_test_excellent = test_r[excellent_qa_indx,:]
quality_assessment_test_excellent = test_qa[excellent_qa_indx,:]


# weighted macro-average across all indivduals

test_metrics = util.collecting_individual_metrics(model_reint, x_test_excellent, p_test_excellent, rhythm_test_excellent, out_message=False)
test_metrics = pd.DataFrame.from_dict(test_metrics).T.rename(columns={0:'TPR', 1:'TNR', 2:'FPR', 3:'FNR', 4:"total_samples"})


for m in ['TPR', 'TNR', 'FPR', 'FNR']:
    metric_wmu = np.average(test_metrics[m][~test_metrics[m].isna()], weights=test_metrics['total_samples'][~test_metrics[m].isna()])
    print('%s: %0.2f' % (m, metric_wmu))
    
# PPV, NPV and F1

episode_m = util.episode_metrics(model_reint, x_test_excellent, p_test_excellent, rhythm_test_excellent, out_message=False)
episode_metrics = pd.DataFrame(episode_m).T
episode_metrics.rename(columns={0:'TPR', 1:'TNR', 2:'PPV', 3:'NPV', 4:"FPR", 5:'FNR', 6:'F1', 7:'total_samples'}, inplace=True)
for m in ['PPV', 'NPV', 'F1']:
    print('%s: %0.2f' % (m, episode_metrics[m]))


TPR: 0.97
TNR: 0.89
FPR: 0.11
FNR: 0.03
PPV: 0.55
NPV: 1.00
F1: 0.71


## DeepBeat Multi-task (pretrained with CDAE + AF + Excellent QA)

#### Loading pretrained DeepBeat model

In [11]:
path_to_model ='/path/to/model/'
model_name = 'deepbeat.h5'
deepbeat = load_model(path_to_model + model_name) 

#### DeepBeat Multi-task (pretrain CDAE + AF + Excellent QA) 


In [12]:
## QA results

predictions_qa, predictions_r = deepbeat.predict(test_x)
predictions_QA = np.argmax(predictions_qa, axis=1)

#print(classification_report(np.argmax(test_qa, axis=1), predictions_QA))


excellent_qa_indx = np.where(predictions_QA==2)[0]
x_test_excellent = test_x[excellent_qa_indx,:]
p_test_excellent = test_p.iloc[excellent_qa_indx,:]
rhythm_test_excellent = test_r[excellent_qa_indx,:]
quality_assessment_test_excellent = test_qa[excellent_qa_indx,:]


# weighted macro-average across all indivduals

test_metrics = util.collecting_individual_metrics(deepbeat, x_test_excellent, p_test_excellent, rhythm_test_excellent, out_message=False)
test_metrics = pd.DataFrame.from_dict(test_metrics).T.rename(columns={0:'TPR', 1:'TNR', 2:'FPR', 3:'FNR', 4:"total_samples"})


for m in ['TPR', 'TNR', 'FPR', 'FNR']:
    metric_wmu = np.average(test_metrics[m][~test_metrics[m].isna()], weights=test_metrics['total_samples'][~test_metrics[m].isna()])
    print('%s: %0.2f' % (m, metric_wmu))
    
# PPV, NPV and F1

episode_m = util.episode_metrics(deepbeat, x_test_excellent, p_test_excellent, rhythm_test_excellent, out_message=False)
episode_metrics = pd.DataFrame(episode_m).T
episode_metrics.rename(columns={0:'TPR', 1:'TNR', 2:'PPV', 3:'NPV', 4:"FPR", 5:'FNR', 6:'F1', 7:'total_samples'}, inplace=True)
for m in ['PPV', 'NPV', 'F1']:
    print('%s: %0.2f' % (m, episode_metrics[m]))

TPR: 0.98
TNR: 0.99
FPR: 0.01
FNR: 0.02
PPV: 0.94
NPV: 1.00
F1: 0.96
